In [ ]:
Main

In [ ]:
import tkinter as tk
from tkinter import Menu
from tkinter import ttk, filedialog, messagebox 
import sqlite3
import csv
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

#SQL
conn = sqlite3.connect(":memory:")
c = conn.cursor()

sql_create_keszlet_table = """ CREATE TABLE IF NOT EXISTS keszlet (
                                    id integer PRIMARY KEY,
                                    raktar_id integer NOT NULL,
                                    termek_id integer NOT NULL,
                                    mennyiseg integer NOT NULL
                                ); """

c = conn.cursor()
c.execute(sql_create_keszlet_table)

sql_create_raktar_table = """ CREATE TABLE IF NOT EXISTS raktar(
                                    id integer PRIMARY KEY,
                                    rnev text NOT NULL,
                                    helyszin text NOT NULL,
                                    kapacitas integer NOT NULL
                                ); """

c = conn.cursor()
c.execute(sql_create_raktar_table)

sql_create_termek_table = """ CREATE TABLE IF NOT EXISTS termek (
                                    id integer PRIMARY KEY,
                                    jnev text NOT NULL,
                                    mezszam integer NOT NULL,
                                    csapat text NOT NULL,
                                    ar integer NOT NULL
                                ); """

c.execute(sql_create_termek_table)


def insert_into_keszlet(conn, keszlet):
    sql = ''' INSERT INTO keszlet(raktar_id,termek_id,mennyiseg)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, keszlet)
    conn.commit()
    
def insert_into_raktar(conn, raktar):
    sql = ''' INSERT INTO raktar(rnev,helyszin,kapacitas)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, raktar)
    conn.commit()

def insert_into_termek(conn, termek):
    sql = ''' INSERT INTO termek(jnev,mezszam,csapat,ar)
              VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, termek)
    conn.commit()


sql1 = """SELECT * FROM keszlet INNER JOIN termek on keszlet.termek_id = termek.id"""
sql2 = """SELECT * FROM keszlet INNER JOIN raktar on keszlet.raktar_id = raktar.id"""

cur1 = conn.cursor()
cur1.execute(sql1)
cur2 = conn.cursor()
cur2.execute(sql2)

textbox_jnev = None
textbox_mezszam = None
textbox_csapat = None
textbox_ar = None

textbox_rnev = None
textbox_helyszin = None
textbox_kapacitas = None

textbox_raktar_id = None
textbox_termek_id = None
textbox_mennyiseg = None

selected_record_id = None

choice = None


#fuggvenyek
def create_treeview(frame, headers):
    tree_style = ttk.Style()
    tree = ttk.Treeview(frame, columns=headers, show='headings')
    for header in headers:
        tree_style.configure("Treeview.Heading", font=("Georgia", 15, "bold"), )
        tree_style.configure("Treeview", background="lightgrey", font=("Arial", 13,),  rowheight=30)
        tree.heading(header, text=header)
        tree.column(header, width=155)  

    return tree

def populate_treeview(tree, cursor):
    for row in cursor.fetchall():
        tree.insert("", "end", values=row)

     
def is_valid_number(input_str):
    try:
        int(input_str)
        return True
    except ValueError:
        return False

def is_valid_text(input_str):
    return input_str.isalpha()


def new_rec_termek():
    global textbox_jnev, textbox_mezszam, textbox_csapat, textbox_ar
    new_record_window = tk.Toplevel(root)
    new_record_window.title("New data")

    label_jnev = tk.Label(new_record_window, text="Name:")
    label_jnev.grid(row=0, column=0, padx=5)
    tk.Label(new_record_window, text="*", fg="red").grid(row=0, column=2, sticky="w")

    validate_text = new_record_window.register(is_valid_text)
    textbox_jnev = tk.Entry(new_record_window, width=20, validate="key", validatecommand=(validate_text, "%S"))
    textbox_jnev.grid(row=0, column=1, padx=5)

    label_mezszam = tk.Label(new_record_window, text="Jersey:")
    label_mezszam.grid(row=0, column=3, padx=5)
    tk.Label(new_record_window, text="*", fg="red").grid(row=0, column=5, sticky="w")

    validate_number = new_record_window.register(is_valid_number)
    textbox_mezszam = tk.Entry(new_record_window, width=20, validate="key", validatecommand=(validate_number, "%S"))
    textbox_mezszam.grid(row=0, column=4, padx=5)

    label_csapat = tk.Label(new_record_window, text="Club:")
    label_csapat.grid(row=1, column=0, padx=5)
    tk.Label(new_record_window, text="*", fg="red").grid(row=1, column=2, sticky="w")

    textbox_csapat = tk.Entry(new_record_window, width=20, validate="key", validatecommand=(validate_text, "%S"))
    textbox_csapat.grid(row=1, column=1, padx=5)

    label_ar = tk.Label(new_record_window, text="Price:")
    label_ar.grid(row=1, column=3, padx=5)
    tk.Label(new_record_window, text="*", fg="red").grid(row=1, column=5, sticky="w")

    textbox_ar = tk.Entry(new_record_window, width=20, validate="key", validatecommand=(validate_number, "%S"))
    textbox_ar.grid(row=1, column=4, padx=5)

    def insert_record():
        global termek_tree
        if not all((textbox_jnev.get(), textbox_mezszam.get(), textbox_csapat.get(), textbox_ar.get())):
            messagebox.showwarning("Missing Data", "All fields are required!")
        else:
            
            c.execute("SELECT * FROM termek WHERE jnev=? AND mezszam=? AND csapat=? AND ar=?", 
                      (textbox_jnev.get(), int(textbox_mezszam.get()), textbox_csapat.get(), int(textbox_ar.get())))
            existing_record = c.fetchone()

            if existing_record:
                messagebox.showwarning("Already Exists", "This record already exists!")

            else:
                insert_into_termek(conn, (textbox_jnev.get(), int(textbox_mezszam.get()), textbox_csapat.get(), int(textbox_ar.get())))
                refresh_treeview("PRODUCT_GUI") 
        clear_textboxes()


    insert_button = tk.Button(new_record_window, text="Add", command=insert_record)
    insert_button.grid(row=0, column=6, rowspan=2, padx=5)

    
    tk.Label(new_record_window, text="*This field is required!", fg="red").grid(row=2, column=0, columnspan=7, pady=(10, 5))

    

def new_rec_keszlet():
    global textbox_raktar_id, textbox_termek_id, textbox_mennyiseg
    new_record_window = tk.Toplevel(root)
    new_record_window.title("New data")

    label_raktar_id = tk.Label(new_record_window, text="Warehouse ID:")
    label_raktar_id.grid(row=0, column=0, padx=5)
    tk.Label(new_record_window, text="*", fg="red").grid(row=0, column=2, sticky="w")

    validate_number = new_record_window.register(is_valid_number)
    textbox_raktar_id = tk.Entry(new_record_window, width=20, validate="key", validatecommand=(validate_number, "%S"))
    textbox_raktar_id.grid(row=0, column=1, padx=5)

    label_termek_id = tk.Label(new_record_window, text="Product ID:")
    label_termek_id.grid(row=0, column=3, padx=5)
    tk.Label(new_record_window, text="*", fg="red").grid(row=0, column=5, sticky="w")

    textbox_termek_id = tk.Entry(new_record_window, width=20, validate="key", validatecommand=(validate_number, "%S"))
    textbox_termek_id.grid(row=0, column=4, padx=5)

    label_mennyiseg = tk.Label(new_record_window, text="Quantity:")
    label_mennyiseg.grid(row=1, column=0, padx=5)
    tk.Label(new_record_window, text="*", fg="red").grid(row=1, column=2, sticky="w")

    textbox_mennyiseg = tk.Entry(new_record_window, width=20, validate="key", validatecommand=(validate_number, "%S"))
    textbox_mennyiseg.grid(row=1, column=1, padx=5)

    def insert_record():
        global keszlet_tree
        if not all((textbox_raktar_id.get(), textbox_termek_id.get(), textbox_mennyiseg.get())):
            messagebox.showwarning("Missing Data", "All fields are required!")

        else:
            c.execute("SELECT * FROM keszlet WHERE raktar_id=? AND termek_id=? AND mennyiseg=?",
                      (int(textbox_raktar_id.get()), int(textbox_termek_id.get()), int(textbox_mennyiseg.get())))
            existing_record = c.fetchone()
            
            
            if existing_record:
                messagebox.showwarning("Already Exists", "This record already exists!")

            else:
            
                insert_into_keszlet(conn, (int(textbox_raktar_id.get()), int(textbox_termek_id.get()), int(textbox_mennyiseg.get())))
                refresh_treeview("INVENTORY_GUI")
        clear_textboxes1()

    insert_button = tk.Button(new_record_window, text="Add", command=insert_record)
    insert_button.grid(row=0, column=6, rowspan=2, padx=5)

    
    tk.Label(new_record_window, text="*This field is required!", fg="red").grid(row=2, column=0, columnspan=7, pady=(10, 5))


def new_rec_raktar():
    global textbox_rnev, textbox_helyszin, textbox_kapacitas
    new_record_window = tk.Toplevel(root)
    new_record_window.title("New data")

    label_nev = tk.Label(new_record_window, text="Name:")
    label_nev.grid(row=0, column=0, padx=5)
    tk.Label(new_record_window, text="*", fg="red").grid(row=0, column=2, sticky="w")

    validate_text = new_record_window.register(is_valid_text)
    textbox_rnev = tk.Entry(new_record_window, width=20, validate="key", validatecommand=(validate_text, "%S"))
    textbox_rnev.grid(row=0, column=1, padx=5)

    label_helyszin = tk.Label(new_record_window, text="Location:")
    label_helyszin.grid(row=1, column=0, padx=5)
    tk.Label(new_record_window, text="*", fg="red").grid(row=1, column=2, sticky="w")

    textbox_helyszin = tk.Entry(new_record_window, width=20, validate="key", validatecommand=(validate_text, "%S"))
    textbox_helyszin.grid(row=1, column=1, padx=5)

    label_kapacitas = tk.Label(new_record_window, text="Capacity:")
    label_kapacitas.grid(row=2, column=0, padx=5)
    tk.Label(new_record_window, text="*", fg="red").grid(row=2, column=2, sticky="w")

    validate_number = new_record_window.register(is_valid_number)
    textbox_kapacitas = tk.Entry(new_record_window, width=20, validate="key", validatecommand=(validate_number, "%S"))
    textbox_kapacitas.grid(row=2, column=1, padx=5)

    def insert_record():
        global raktar_tree
        if not all((textbox_rnev.get(), textbox_helyszin.get(), textbox_kapacitas.get())):
            messagebox.showwarning("Missing Data", "All fields are required!")
        else:
            c.execute("SELECT * FROM raktar WHERE rnev=? AND helyszin=? AND kapacitas=?",
                      (textbox_rnev.get(), textbox_helyszin.get(), int(textbox_kapacitas.get())))
            existing_record = c.fetchone()
        
            if existing_record:
                messagebox.showwarning("Already Exists", "This record already exists!")

            else:
                insert_into_raktar(conn, (textbox_rnev.get(), textbox_helyszin.get(), int(textbox_kapacitas.get())))
                refresh_treeview("WAREHOUSE_GUI")
        clear_textboxes2()
        
            
    insert_button = tk.Button(new_record_window, text="Add", command=insert_record)
    insert_button.grid(row=3, column=0, columnspan=2, pady=5)
  
    tk.Label(new_record_window, text="*This field is required!", fg="red").grid(row=4, column=0, columnspan=2, pady=(10, 5))

def import_data_termek():
    file_path = filedialog.askopenfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])

    if file_path:
        try:
            with open(file_path, 'r') as csvfile:
                csvreader = csv.reader(csvfile)
                next(csvreader)  

                for row in csvreader:
                    
                    nev, mezsza, csapat, ar = row[1], int(row[2]), row[3], int(row[4])
                    insert_into_termek(conn, (nev, mezsza, csapat, ar))

            refresh_treeview("PRODUCT_GUI")  
            messagebox.showinfo("Import Successful", "The data has been successfully imported.")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred during import: {str(e)}")



def export_data_termek():
    file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])

    if file_path:
        with open(file_path, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(["ID", "Name", "Jersey", "Club", "Price"])

            c.execute("SELECT * FROM termek")
            termek_rows = c.fetchall()

            for row in termek_rows:
                csvwriter.writerow(row)

            messagebox.showinfo("Export Successful", "The data has been successfully exported.")



def import_data_raktar():
    file_path = filedialog.askopenfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])

    if file_path:
        try:
            with open(file_path, 'r') as csvfile:
                csvreader = csv.reader(csvfile)
                next(csvreader)  

                for row in csvreader:
                    
                    nev, helyszin, kapacitas = row[1], row[2], int(row[3])
                    insert_into_raktar(conn, (nev, helyszin, kapacitas))

            refresh_treeview("WAREHOUSE_GUI")  
            messagebox.showinfo("Import Successful", "The data has been successfully imported.")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred during import: {str(e)}")


def export_data_raktar():
    file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])

    if file_path:
        with open(file_path, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(["ID", "Name", "Location", "Capacity"])

            c.execute("SELECT * FROM raktar")
            raktar_rows = c.fetchall()

            for row in raktar_rows:
                csvwriter.writerow(row)

            messagebox.showinfo("Export Successful", "The data has been successfully exported.")


def import_data_keszlet():
    file_path = filedialog.askopenfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])

    if file_path:
        try:
            with open(file_path, 'r') as csvfile:
                csvreader = csv.reader(csvfile)
                next(csvreader)  

                for row in csvreader:
                    
                    raktar_id, termek_id, mennyiseg = int(row[1]), int(row[2]), int(row[3])
                    insert_into_keszlet(conn, (raktar_id, termek_id, mennyiseg))

            refresh_treeview("INVENTORY_GUI")  
            messagebox.showinfo("Import Successful", "The data has been successfully imported.")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred during import: {str(e)}")




def export_data_keszlet():
    file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])

    if file_path:
        with open(file_path, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(["ID", "Warehouse_id", "Product_id", "Quantity"])

            c.execute("SELECT * FROM keszlet")
            keszlet_rows = c.fetchall()

            for row in keszlet_rows:
                csvwriter.writerow(row)

            messagebox.showinfo("Export Successful", "The data has been successfully exported.")


            
def edit_record_termek():
    global selected_record_id
    selected_item = termek_tree.selection()
    if selected_item:
        item_values = termek_tree.item(selected_item, 'values')
        if item_values:
            selected_record_id = item_values[0]  

            
            edit_window = tk.Toplevel(root)
            edit_window.title("Edit")

            label_nev = tk.Label(edit_window, text="Name:")
            label_nev.grid(row=0, column=0, padx=5)
            
            textbox_nev_edit = tk.Entry(edit_window, width=20)
            textbox_nev_edit.grid(row=0, column=1, padx=5)
            textbox_nev_edit.insert(0, item_values[1])

            label_mezszam = tk.Label(edit_window, text="Jersey:")
            label_mezszam.grid(row=0, column=2, padx=5)

            textbox_mezszam_edit = tk.Entry(edit_window, width=20)
            textbox_mezszam_edit.grid(row=0, column=3, padx=5)
            textbox_mezszam_edit.insert(0, item_values[2])

            label_csapat = tk.Label(edit_window, text="Club:")
            label_csapat.grid(row=1, column=0, padx=5)

            textbox_csapat_edit = tk.Entry(edit_window, width=20)
            textbox_csapat_edit.grid(row=1, column=1, padx=5)
            textbox_csapat_edit.insert(0, item_values[3])

            label_ar = tk.Label(edit_window, text="Price:")
            label_ar.grid(row=1, column=2, padx=5)
 
            textbox_ar_edit = tk.Entry(edit_window, width=20)
            textbox_ar_edit.grid(row=1, column=3, padx=5)
            textbox_ar_edit.insert(0, item_values[4])

            save_button = tk.Button(edit_window, text="Enter", command=lambda: save_changes_termek(textbox_nev_edit.get(), textbox_mezszam_edit.get(), textbox_csapat_edit.get(), textbox_ar_edit.get()))
            save_button.grid(row=0, column=4, rowspan=2, padx=5)


def save_changes_termek(new_nev, new_mezszam, new_csapat, new_ar):
    global selected_record_id
    if selected_record_id:
        
        c.execute("UPDATE termek SET jnev=?, mezszam=?, csapat=?, ar=? WHERE id=?", (new_nev, int(new_mezszam), new_csapat, int(new_ar), selected_record_id))
        conn.commit()
        refresh_treeview("PRODUCT_GUI")  
        selected_record_id = None  

def edit_record_raktar():
    global selected_record_id
    selected_item = raktar_tree.selection()
    if selected_item:
        item_values = raktar_tree.item(selected_item, 'values')
        if item_values:
            selected_record_id = item_values[0] 

            
            edit_window = tk.Toplevel(root)
            edit_window.title("Edit")

            label_nev = tk.Label(edit_window, text="Name:")
            label_nev.grid(row=0, column=0, padx=5)

            textbox_nev_edit = tk.Entry(edit_window, width=20)
            textbox_nev_edit.grid(row=0, column=1, padx=5)
            textbox_nev_edit.insert(0, item_values[1])

            label_helyszin = tk.Label(edit_window, text="Location:")
            label_helyszin.grid(row=1, column=0, padx=5)

            textbox_helyszin_edit = tk.Entry(edit_window, width=20)
            textbox_helyszin_edit.grid(row=1, column=1, padx=5)
            textbox_helyszin_edit.insert(0, item_values[2])

            label_kapacitas = tk.Label(edit_window, text="Capacity:")
            label_kapacitas.grid(row=2, column=0, padx=5)

            textbox_kapacitas_edit = tk.Entry(edit_window, width=20)
            textbox_kapacitas_edit.grid(row=2, column=1, padx=5)
            textbox_kapacitas_edit.insert(0, item_values[3])

            save_button = tk.Button(edit_window, text="Enter", command=lambda: save_changes_raktar(textbox_nev_edit.get(), textbox_helyszin_edit.get(), textbox_kapacitas_edit.get()))
            save_button.grid(row=3, column=0, columnspan=2, pady=5)


def save_changes_raktar(new_nev, new_helyszin, new_kapacitas):
    global selected_record_id
    if selected_record_id:
        
        c.execute("UPDATE raktar SET rnev=?, helyszin=?, kapacitas=? WHERE id=?", (new_nev, new_helyszin, int(new_kapacitas), selected_record_id))
        conn.commit()
        refresh_treeview("WAREHOUSE_GUI")  
        selected_record_id = None  

def edit_record_keszlet():
    global selected_record_id
    selected_item = keszlet_tree.selection()
    if selected_item:
        item_values = keszlet_tree.item(selected_item, 'values')
        if item_values:
            selected_record_id = item_values[0]  

            
            edit_window = tk.Toplevel(root)
            edit_window.title("Edit")

            label_raktar_id = tk.Label(edit_window, text="Warehouse ID:")
            label_raktar_id.grid(row=0, column=0, padx=5)

            textbox_raktar_id_edit = tk.Entry(edit_window, width=20)
            textbox_raktar_id_edit.grid(row=0, column=1, padx=5)
            textbox_raktar_id_edit.insert(0, item_values[1])

            label_termek_id = tk.Label(edit_window, text="Product ID:")
            label_termek_id.grid(row=0, column=2, padx=5)

            textbox_termek_id_edit = tk.Entry(edit_window, width=20)
            textbox_termek_id_edit.grid(row=0, column=3, padx=5)
            textbox_termek_id_edit.insert(0, item_values[2])

            label_mennyiseg = tk.Label(edit_window, text="Quantity:")
            label_mennyiseg.grid(row=1, column=0, padx=5)
  
            textbox_mennyiseg_edit = tk.Entry(edit_window, width=20)
            textbox_mennyiseg_edit.grid(row=1, column=1, padx=5)
            textbox_mennyiseg_edit.insert(0, item_values[3])

            save_button = tk.Button(edit_window, text="Enter", command=lambda: save_changes_keszlet(textbox_raktar_id_edit.get(), textbox_termek_id_edit.get(), textbox_mennyiseg_edit.get()))
            save_button.grid(row=0, column=4, rowspan=2, padx=5)


def save_changes_keszlet(new_raktar_id, new_termek_id, new_mennyiseg):
    global selected_record_id
    if selected_record_id:
        
        c.execute("UPDATE keszlet SET raktar_id=?, termek_id=?, mennyiseg=? WHERE id=?", (int(new_raktar_id), int(new_termek_id), int(new_mennyiseg), selected_record_id))
        conn.commit()
        refresh_treeview("INVENTORY_GUI")  
        selected_record_id = None  

def sort_termek():
    sort_window = tk.Toplevel(root)
    sort_window.title("SORT")
    sort_window.geometry("400x300")
    sort_window.configure(bg="orange")

    def sort_by_column(order, column):
        c.execute(f"SELECT * FROM termek ORDER BY {column} {order}")
        sorted_rows = c.fetchall()
        termek_tree.delete(*termek_tree.get_children())  
        for row in sorted_rows:
            termek_tree.insert("", "end", values=row)

    # Létrehozunk egy középre helyezett frame-et
    content_frame = tk.Frame(sort_window, bg="orange")
    content_frame.place(relx=0.5, rely=0.5, anchor="center")  # Középre helyezés

    # Gombok ebbe a frame-be kerülnek
    sort_by_nev_button = tk.Button(content_frame, text="Sort by Name (Ascending)", command=lambda: sort_by_column("ASC", "jnev"))
    sort_by_nev_button.grid(row=0, column=0, pady=5)

    sort_by_nev_desc_button = tk.Button(content_frame, text="Sort by Name (Descending)", command=lambda: sort_by_column("DESC", "jnev"))
    sort_by_nev_desc_button.grid(row=0, column=1, pady=5)

    sort_by_mezszam_button = tk.Button(content_frame, text="Sort by Jersey (Ascending)", command=lambda: sort_by_column("ASC", "mezszam"))
    sort_by_mezszam_button.grid(row=1, column=0, pady=5)

    sort_by_mezszam_desc_button = tk.Button(content_frame, text="Sort by Jersey (Descending)", command=lambda: sort_by_column("DESC", "mezszam"))
    sort_by_mezszam_desc_button.grid(row=1, column=1, pady=5)

    sort_by_csapat_button = tk.Button(content_frame, text="Sort by Club (Ascending)", command=lambda: sort_by_column("ASC", "csapat"))
    sort_by_csapat_button.grid(row=2, column=0, pady=5)

    sort_by_csapat_desc_button = tk.Button(content_frame, text="Sort by Club (Descending)", command=lambda: sort_by_column("DESC", "csapat"))
    sort_by_csapat_desc_button.grid(row=2, column=1, pady=5)

    sort_by_ar_button = tk.Button(content_frame, text="Sort by Price (Ascending)", command=lambda: sort_by_column("ASC", "ar"))
    sort_by_ar_button.grid(row=3, column=0, pady=5)

    sort_by_ar_desc_button = tk.Button(content_frame, text="Sort by Price (Descending)", command=lambda: sort_by_column("DESC", "ar"))
    sort_by_ar_desc_button.grid(row=3, column=1, pady=5)

    sort_by_original_order_button = tk.Button(content_frame, text="Reset to Original", command=lambda: sort_by_column("ASC", "id"), width=40, height=2)
    sort_by_original_order_button.grid(row=4, column=0, columnspan=2, pady=5)




def sort_raktar():
    sort_window = tk.Toplevel(root)
    sort_window.title("SORT")
    sort_window.geometry("400x300")
    sort_window.configure(bg="orange")

    def sort_by_column(order, column):
        c.execute(f"SELECT * FROM raktar ORDER BY {column} {order}")
        sorted_rows = c.fetchall()
        raktar_tree.delete(*raktar_tree.get_children())  
        for row in sorted_rows:
            raktar_tree.insert("", "end", values=row)

    # Létrehozunk egy középre helyezett frame-et
    content_frame = tk.Frame(sort_window, bg="orange")
    content_frame.place(relx=0.5, rely=0.5, anchor="center")  # Középre helyezés

    sort_by_nev_button = tk.Button(content_frame, text="Sort by Name (Ascending)", command=lambda: sort_by_column("ASC", "rnev"))
    sort_by_nev_button.grid(row=0, column=0, pady=5)
    
    sort_by_nev_desc_button = tk.Button(content_frame, text="Sort by Name (Descending)", command=lambda: sort_by_column("DESC", "rnev"))
    sort_by_nev_desc_button.grid(row=0, column=1, pady=5)
    
    sort_by_helyszin_button = tk.Button(content_frame, text="Sort by Location (Ascending)", command=lambda: sort_by_column("ASC", "helyszin"))
    sort_by_helyszin_button.grid(row=1, column=0, pady=5)

    sort_by_helyszin_desc_button = tk.Button(content_frame, text="Sort by Location (Descending)", command=lambda: sort_by_column("DESC", "helyszin"))
    sort_by_helyszin_desc_button.grid(row=1, column=1, pady=5)

    sort_by_kapacitas_button = tk.Button(content_frame, text="Sort by Capacity (Ascending)", command=lambda: sort_by_column("ASC", "kapacitas"))
    sort_by_kapacitas_button.grid(row=2, column=0, pady=5)

    sort_by_kapacitas_desc_button = tk.Button(content_frame, text="Sort by Capacity (Descending)", command=lambda: sort_by_column("DESC", "kapacitas"))
    sort_by_kapacitas_desc_button.grid(row=2, column=1, pady=5)

    sort_by_original_order_button = tk.Button(content_frame, text="Reset to Original", command=lambda: sort_by_column("ASC", "id"), width=40, height=2)
    sort_by_original_order_button.grid(row=3, column=0, columnspan=2, pady=5)
    
def sort_keszlet():
    sort_window = tk.Toplevel(root)
    sort_window.title("SORT")
    sort_window.geometry("400x300")
    sort_window.configure(bg="orange")

    def sort_by_column(order, column):
        c.execute(f"SELECT * FROM keszlet ORDER BY {column} {order}")
        sorted_rows = c.fetchall()
        keszlet_tree.delete(*keszlet_tree.get_children())  
        for row in sorted_rows:
            keszlet_tree.insert("", "end", values=row)

    # Létrehozunk egy középre helyezett frame-et
    content_frame = tk.Frame(sort_window, bg="orange")
    content_frame.place(relx=0.5, rely=0.5, anchor="center")  # Középre helyezés

    sort_by_raktar_id_button = tk.Button(content_frame, text="Sort by Warehouse ID (Ascending)", command=lambda: sort_by_column("ASC", "raktar_id"))
    sort_by_raktar_id_button.grid(row=0, column=0, pady=5)

    sort_by_raktar_id_desc_button = tk.Button(content_frame, text="Sort by Warehouse ID (Descending)", command=lambda: sort_by_column("DESC", "raktar_id"))
    sort_by_raktar_id_desc_button.grid(row=0, column=1, pady=5)

    sort_by_termek_id_button = tk.Button(content_frame, text="Sort by Product ID (Ascending)", command=lambda: sort_by_column("ASC", "termek_id"))
    sort_by_termek_id_button.grid(row=1, column=0, pady=5)

    sort_by_termek_id_desc_button = tk.Button(content_frame, text="Sort by Product ID (Descending)", command=lambda: sort_by_column("DESC", "termek_id"))
    sort_by_termek_id_desc_button.grid(row=1, column=1, pady=5)

    sort_by_mennyiseg_button = tk.Button(content_frame, text="Sort by Quantity (Ascending)", command=lambda: sort_by_column("ASC", "mennyiseg"))
    sort_by_mennyiseg_button.grid(row=2, column=0, pady=5)

    sort_by_mennyiseg_desc_button = tk.Button(content_frame, text="Sort by Quantity (Descending)", command=lambda: sort_by_column("DESC", "mennyiseg"))
    sort_by_mennyiseg_desc_button.grid(row=2, column=1, pady=5)

    sort_by_original_order_button = tk.Button(content_frame, text="Reset to Original", command=lambda: sort_by_column("ASC", "id"), width=40, height=2)
    sort_by_original_order_button.grid(row=3, column=0, columnspan=2, pady=5)



def delete_rec_termek():
    global termek_tree, textbox_jnev, textbox_mezszam, textbox_csapat, textbox_ar

    selected_item = termek_tree.selection()
    item_values = termek_tree.item(selected_item, 'values')
    if item_values:
        c.execute("DELETE FROM termek WHERE id=?", (item_values[0],))
        conn.commit()
        refresh_treeview("PRODUCT_GUI")
              

def delete_rec_keszlet():
    global keszlet_tree, textbox_raktar_id, textbox_termek_id, textbox_mennyiseg

    selected_item = keszlet_tree.selection()
    item_values = keszlet_tree.item(selected_item, 'values')
    if item_values:
        c.execute("DELETE FROM keszlet WHERE id=?", (item_values[0],))
        conn.commit()
        refresh_treeview("INVENTORY_GUI")
               

def delete_rec_raktar():
    global raktar_tree, textbox_rnev, textbox_helyszin, textbox_kapacitas

    selected_item = raktar_tree.selection()
    item_values = raktar_tree.item(selected_item, 'values')
    if item_values:
        c.execute("DELETE FROM raktar WHERE id=?", (item_values[0],))
        conn.commit()
        refresh_treeview("WAREHOUSE_GUI")


def open_sum_kimutatas_gui():
    
    c.execute("SELECT ar FROM termek")
    ar_values = c.fetchall()

    
    sum_result = sum(ar[0] for ar in ar_values)
    
    sum_text = tk.Label(sum_kimutatas_window, text="Total value of jerseys: ", font=("Arial", 24), bg="orange", fg="black")
    sum_text.pack(pady=20)
    euro_text = tk.Label(sum_kimutatas_window, text=f"{sum_result} EURO", font=("Arial", 30), bg="lightorange", fg="black", justify="center", anchor="center")
    euro_text.pack(expand=True)


#ablak

def open_new_window(choice):
    global termek_tree, keszlet_tree, raktar_tree
    
    new_window = tk.Toplevel(root)
    new_window.title(choice)
    new_window.geometry("780x400")
    new_window.configure(bg="white")

    
    
    #######################################################################termek###################################################################
    if choice == "PRODUCT_GUI":
        label = tk.Label(new_window, text="Product Table", font=("Arial", 24), bg='orange', fg="black")
        label.pack(pady=20)
        new_window.configure(bg="orange")
        
        button_frame = tk.Frame(new_window, bg='orange')
        button_frame.pack()
        
        buttons = ['➕ New', '✏ Edit', '🔀 Sort', '📂 Import', '💾 Export', '🗑️ Delete']

        for btn_text in buttons:
            if btn_text == '➕ New':
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='green', fg="white", padx=10, pady=5, command=new_rec_termek)
            elif btn_text == '✏ Edit':
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='green', fg="white", padx=10, pady=5, command=edit_record_termek)
            elif btn_text == '🔀 Sort':
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='green', fg="white", padx=10, pady=5, command=sort_termek)
            elif btn_text == '📂 Import':
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='green', fg="white", padx=10, pady=5, command=import_data_termek)
            elif btn_text == '💾 Export':
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='green', fg="white", padx=10, pady=5, command=export_data_termek)
            elif btn_text == '🗑️ Delete':
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='red', fg="white", padx=10, pady=5, command=delete_rec_termek)  
            else:
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='red', fg="black", padx=10, pady=5)
            btn.pack(side=tk.LEFT, padx=10)
        
        termek_headers = ['ID', 'Name', 'Jersey', 'Club', 'Price']
        termek_tree = create_treeview(new_window, termek_headers)
        termek_tree.pack(pady=20)
    
        
           
        
        
    ######################################################################keszlet###############################################################
    elif choice == "INVENTORY_GUI":
        label = tk.Label(new_window, text="Inventory Table", font=("Arial", 24), bg='orange', fg="black")
        label.pack(pady=20)
        new_window.configure(bg="orange")
        
        button_frame = tk.Frame(new_window, bg='orange')
        button_frame.pack()
    
        buttons = ['➕ New', '✏ Edit', '🔀 Sort', '📂 Import', '💾 Export', '🗑️ Delete']

        for btn_text in buttons:
            if btn_text == '➕ New':
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='green', fg="white", padx=10, pady=5, command=new_rec_keszlet)
            elif btn_text == '✏ Edit':
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='green', fg="white", padx=10, pady=5, command=edit_record_keszlet)
            elif btn_text == '🔀 Sort':
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='green', fg="white", padx=10, pady=5, command=sort_keszlet)
            elif btn_text == '📂 Import':
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='green', fg="white", padx=10, pady=5, command=import_data_keszlet)
            elif btn_text == '💾 Export':
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='green', fg="white", padx=10, pady=5, command=export_data_keszlet)
            elif btn_text == '🗑️ Delete':
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='red', fg="white", padx=10, pady=5, command=delete_rec_keszlet)  
                
            
            else:
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), padx=10, pady=5)
            btn.pack(side=tk.LEFT, padx=10)
        
        
        keszlet_headers = ['ID', 'Warehouse ID', 'Product ID', 'Quantity']
        keszlet_tree = create_treeview(new_window, keszlet_headers)
        keszlet_tree.pack(pady=20)
    

        
    ###################################################################raktar#################################################################
    elif choice == "WAREHOUSE_GUI":
        label = tk.Label(new_window, text="Warehouse Table", font=("Arial", 24), bg='orange', fg="black")
        label.pack(pady=20)
        new_window.configure(bg="orange")
        
        button_frame = tk.Frame(new_window, bg='orange')
        button_frame.pack()

        buttons = ['➕ New', '✏ Edit', '🔀 Sort', '📂 Import', '💾 Export', '🗑️ Delete']

        for btn_text in buttons:
            if btn_text == '➕ New':
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='green', fg="white", padx=10, pady=5, command=new_rec_raktar)
            elif btn_text == '✏ Edit':
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='green', fg="white", padx=10, pady=5, command=edit_record_raktar)
            elif btn_text == '🔀 Sort':
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='green', fg="white", padx=10, pady=5, command=sort_raktar)
            elif btn_text == '📂 Import':
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='green', fg="white", padx=10, pady=5, command=import_data_raktar)
            elif btn_text == '💾 Export':
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='green', fg="white", padx=10, pady=5, command=export_data_raktar)
            elif btn_text == '🗑️ Delete':
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), bg='red', fg="white", padx=10, pady=5, command=delete_rec_raktar)  
                
            
            else:
                btn = tk.Button(button_frame, text=btn_text, font=("Arial", 12), padx=10, pady=5)
            btn.pack(side=tk.LEFT, padx=10)
        
        raktar_headers = ['ID', 'Name', 'Location', 'Capacity']
        raktar_tree = create_treeview(new_window, raktar_headers)
        raktar_tree.pack(pady=20)
        
###################################################################kimutatas1######################################################################
        
    elif choice == "Report 1":
        new_window.configure(bg="orange")
        
        c.execute("SELECT COUNT(*) FROM termek")
        count = c.fetchone()[0]
        if count > 0:
            new_window.configure(bg="orange")
            c.execute("SELECT ar FROM termek")
            ar_values = c.fetchall()

            sum_result = sum(ar[0] for ar in ar_values)

            formatted_sum = f"{sum_result:,.0f}".replace(",", ".")
            
            sum_text = tk.Label(new_window, text="Total value of jerseys: ", font=("Georgia", 30), bg="orange", fg="black")
            sum_text.pack(pady=20)
            euro_text = tk.Label(new_window, text=f"{formatted_sum} €", font=("Times New Roman", 34), bg="orange", fg="black")
            euro_text.pack(expand=True)
        else:
            tk.Label(new_window, text="No data in the product table!", font=("Arial", 25), bg="orange", fg="red").pack(pady=20)
            

          




###################################################################kimutatas2######################################################################
        
        
    elif choice == "Report 2":
        new_window.configure(bg="orange")
        
        c.execute("SELECT COUNT(*) FROM raktar")
        count1 = c.fetchone()[0]
        c.execute("SELECT COUNT(*) FROM termek")
        count2 = c.fetchone()[0]
        c.execute("SELECT COUNT(*) FROM keszlet")
        count3 = c.fetchone()[0]
        
        if count1 > 0 and count2 > 0 and count3 > 0:
            c.execute("SELECT max(ar) FROM termek")
            maximum = c.fetchone()[0]
            
            max_id_result = c.execute("SELECT id FROM termek WHERE ar=?", (maximum,)).fetchone()
            max_nev_result = c.execute("SELECT jnev FROM termek WHERE ar=?", (maximum,)).fetchone()
            max_mez_result = c.execute("SELECT mezszam FROM termek WHERE ar=?", (maximum,)).fetchone()
            max_csapat_result = c.execute("SELECT csapat FROM termek WHERE ar=?", (maximum,)).fetchone()
            
            if max_id_result and max_nev_result and max_mez_result and max_csapat_result:
                max_id = max_id_result[0]
                max_nev = max_nev_result[0]
                max_mez = max_mez_result[0]
                max_csapat = max_csapat_result[0]
                
                result = c.execute("SELECT raktar_id FROM keszlet WHERE termek_id=?", (max_id,)).fetchone()
                if result:
                    hol_van = result[0]
                    hol_van_result = c.execute("SELECT helyszin FROM raktar WHERE id=?", (hol_van,)).fetchone()
                    hol_van2 = hol_van_result[0] if hol_van_result else "No location available"
                else:
                    hol_van2 = "-"
                
                count_text = tk.Label(new_window, text="The most valuable jersey in stock:", font=("Georgia", 25), bg="orange", fg="black")
                count_text.pack(pady=20)
                
                count_value_text = tk.Label(new_window, text=f"{max_nev}  {max_mez}  {max_csapat}  {maximum} €", font=("Times New Roman", 25, 'bold'), bg="orange", fg="black", justify="center", anchor="center")
                count_value_text.pack(expand=True)
                
                db_value_text = tk.Label(new_window, text=f"The warehouse in {hol_van2} has it in stock.", font=("Georgia", 25), bg="orange", fg="black", justify="center", anchor="center")
                db_value_text.pack(expand=True)
            
            else:
                tk.Label(new_window, text="No data in the required tables!", font=("Arial", 25), bg="orange", fg="red").pack(pady=20)
                hol_van2 = c.execute("SELECT helyszin FROM raktar WHERE id=?", (hol_van,)).fetchone()[0]
                    

                count_text = tk.Label(new_window, text="The most valuable jersey in stock: ", font=("Georgia", 25), bg="orange", fg="black")
                count_text.pack(pady=20)

                count_value_text = tk.Label(new_window, text=f"{max_nev}  {max_mez}  {max_csapat}  {maximum} €", font=("Times New Roman", 25), bg="orange", fg="black", justify="center", anchor="center")
                count_value_text.pack(expand=True)

                
                db_value_text = tk.Label(new_window, text=f"The warehouse in {hol_van2} has it in stock.", font=("Georgia", 25), bg="orange", fg="black", justify="center", anchor="center")
                db_value_text.pack(expand=True)
        else:
            tk.Label(new_window, text="No data in the required tables!", font=("Arial", 25), bg="orange", fg="red").pack(pady=20)
            
            
###################################################################kimutatas3######################################################################

        
    elif choice == "Report 3":
        new_window.configure(bg="orange")
        label = tk.Label(new_window, text="Select a warehouse:", font=("Georgia", 30), bg="orange", fg="black")
        label.pack(pady=20)

    
        c.execute("SELECT COUNT(*) FROM raktar")
        count = c.fetchone()[0]
        if count > 0:
            
            c.execute("SELECT DISTINCT rnev FROM raktar")
            nevek = [row[0] for row in c.fetchall()]

            selected_nev = tk.StringVar()
           
            lenyilo_lista = ttk.Combobox(new_window, textvariable=selected_nev, values=nevek)
            lenyilo_lista.pack(pady=10)
            
        
            def kivalasztas_esemeny(event):
                nev = selected_nev.get()
                if nev:
                    kapacitas = c.execute("SELECT kapacitas FROM raktar WHERE rnev=?", (nev,)).fetchone()[0]
                    id = c.execute("SELECT id FROM raktar WHERE rnev=?", (nev,)).fetchone()[0]
                    mennyiseg = c.execute("SELECT SUM(mennyiseg) FROM keszlet WHERE raktar_id=?", (id,)).fetchone()[0]
                    hely = c.execute("SELECT helyszin FROM raktar WHERE rnev=?", (nev,)).fetchone()[0]
                    
                  
                    eredmeny_label.config(text=f"Stock in the warehouse of {hely}: {mennyiseg}")
                    eredmeny_label2.config(text=f"Capacity of the warehouse in {hely}: {kapacitas}")

                              
                    
                    chart_window = tk.Toplevel()
                    chart_window.title("Warehouse Capacity Rate")               
                    fig = Figure(figsize=(4, 3), dpi=80)
                    ax = fig.add_subplot(111)             
                    data = [mennyiseg]                   
                    bars = ax.bar(range(len(data)), data, color='orange')                
                    ax.bar(range(len(data)), data, color='orange')                   
                    ax.set_ylim(0, kapacitas)                   
                    ax.set_xticks([])

                    for bar in bars:
                        height = bar.get_height()
                        ax.text(bar.get_x() + bar.get_width() / 2, height,
                                f'{(height / kapacitas) * 100:.2f}%', ha='center', va='bottom')

                    
                    canvas = FigureCanvasTkAgg(fig, master=chart_window)
                    canvas_widget = canvas.get_tk_widget()                 
                    canvas_widget.pack(side=tk.TOP, fill=tk.BOTH, expand=1)
                    

            lenyilo_lista.bind("<<ComboboxSelected>>", kivalasztas_esemeny)

            eredmeny_label = tk.Label(new_window, text="", font=("Georgia", 30), bg="orange", fg="black")
            eredmeny_label.pack(pady=20)
            eredmeny_label2 = tk.Label(new_window, text="", font=("Georgia", 30), bg="orange", fg="black")
            eredmeny_label2.pack(pady=20)
             
        else:         
            tk.Label(new_window, text="No available warehouse in the table!", font=("Arial", 25), bg="orange", fg="red").pack(pady=20)

def clear_textboxes():
    username_entry.delete(0, tk.END)
    password_entry.delete(0, tk.END)
    
    
def refresh_treeview(choice):
    tree = None
    if choice == 'PRODUCT_GUI':
        tree = termek_tree
        cursor = cur1
        cursor.execute("SELECT * FROM termek")
        
    elif choice == 'INVENTORY_GUI':
        tree = keszlet_tree
        cursor = cur1
        cursor.execute("SELECT * FROM keszlet")
        
    elif choice == 'WAREHOUSE_GUI':
        tree = raktar_tree
        cursor = cur2
        cursor.execute("SELECT * FROM raktar")
        
    
    for item in tree.get_children():
        tree.delete(item)
  
    populate_treeview(tree, cursor)
  
def clear_textboxes():
    global textbox_jnev, textbox_mezszam, textbox_csapat, textbox_ar
    if textbox_jnev:
        textbox_jnev.delete(0, tk.END)
    if textbox_mezszam:
        textbox_mezszam.delete(0, tk.END)
    if textbox_csapat:
        textbox_csapat.delete(0, tk.END)
    if textbox_ar:
        textbox_ar.delete(0, tk.END)

        
def clear_textboxes1():
    global textbox_raktar_id, textbox_termek_id, textbox_mennyiseg
    if textbox_raktar_id:
        textbox_raktar_id.delete(0, tk.END)
    if textbox_termek_id:
        textbox_termek_id.delete(0, tk.END)    
    if textbox_mennyiseg:
        textbox_mennyiseg.delete(0, tk.END)
        
def clear_textboxes2():
    global textbox_rnev, textbox_helyszin, textbox_kapacitas
    if textbox_rnev:
        textbox_rnev.delete(0, tk.END)
    if textbox_helyszin:
        textbox_helyszin.delete(0, tk.END)    
    if textbox_kapacitas:
        textbox_kapacitas.delete(0, tk.END)


def login(event=None): 
    global login_frame, username_entry, password_entry, login_button, error_label, menu_bar
   
    if not username_entry.get() and not password_entry.get():
        error_label.config(text="The username and password fields are empty!")
    elif not username_entry.get():
        error_label.config(text="The username field is empty!")
    elif not password_entry.get():
        error_label.config(text="The password field is empty!")
    elif username_entry.get() == "a" and password_entry.get() == "a":
        
        clear_textboxes()
        error_label.config(text="")
        
        login_frame.destroy()
        
        menu_bar = Menu(root)
        root.config(menu=menu_bar)
        
        file_menu = Menu(menu_bar, tearoff=0)
        menu_bar.add_cascade(label="Tables", menu=file_menu)

        
        file_menu.add_command(label="Product GUI", command=lambda: open_new_window("PRODUCT_GUI"))
        file_menu.add_command(label="Inventory GUI", command=lambda: open_new_window("INVENTORY_GUI"))
        file_menu.add_command(label="Warehouse GUI", command=lambda: open_new_window("WAREHOUSE_GUI"))

        
        report_menu = Menu(menu_bar, tearoff=0)
        menu_bar.add_cascade(label="Reports", menu=report_menu)

        
        report_menu.add_command(label="Report1", command=lambda: open_new_window("Report 1"))
        report_menu.add_command(label="Report2", command=lambda: open_new_window("Report 2"))
        report_menu.add_command(label="Report3", command=lambda: open_new_window("Report 3"))
        
        welcome_label = tk.Label(root, text="Welcome to the main window!", font=("Georgia", 30,), bg='orange', fg='black')
        welcome_label.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

    elif username_entry.get() == "admin":    
        error_label.config(text="The password is incorrect, try again!")
        
        clear_textboxes()
        
    elif password_entry.get() == "admin":
        
        error_label.config(text="The username is incorrect, try again!")

        clear_textboxes()

    else:
        error_label.config(text="The username and password are incorrect, try again!")

        clear_textboxes()

#foablak
root = tk.Tk()
root.title("Football Jersey Software")
root.geometry("600x600")
root.configure(bg='orange')

login_frame = tk.Frame(root, bg="orange")
login_frame.pack(expand=True, padx=50, pady=100)

tk.Label(login_frame, text="Username:", bg="orange", font=("Arial", 16, 'bold')).grid(row=0, column=0, padx=10, pady=10)
username_entry = tk.Entry(login_frame, font=("Arial", 16), width=15)
username_entry.grid(row=0, column=1, padx=10, pady=10)

tk.Label(login_frame, text="Password:", bg="orange", font=("Arial", 16, 'bold')).grid(row=1, column=0, padx=10, pady=10)
password_entry = tk.Entry(login_frame, show="*", font=("Arial", 16), width=15)
password_entry.grid(row=1, column=1, padx=10, pady=10)

login_button = tk.Button(login_frame, text="Login", command=login, font=("Arial", 16, 'bold'), bg='green', fg='white')
login_button.grid(row=2, column=0, columnspan=2, pady=20)

error_label = tk.Label(login_frame, text="", fg="red", bg="orange", font=("Arial", 10, 'bold'))
error_label.grid(row=3, column=0, columnspan=2)

root.bind("<Return>", login)


#run
root.mainloop()






